In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [3]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [4]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 1.38 s, sys: 143 ms, total: 1.52 s
Wall time: 11 s


576

In [5]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 1.67 s, sys: 113 ms, total: 1.78 s
Wall time: 6.42 s


2

In [13]:
Eck_points_perms = []
for proj in simm_SE1:
    Eck_points_perms.append(Permutation(SE1.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(1, '1')

In [14]:
lines_perms = []
for simm in simm_SE1:
    lines_perms.append(Permutation(SE1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(2, 'C2')